In [1]:
pip install transformers==4.46.2


Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install \
  torch==2.4.1+cu121 \
  torchvision==0.19.1+cu121 \
  torchaudio==2.4.1+cu121 \
  --index-url https://download.pytorch.org/whl/cu121 \
  transformers==4.45.2 \
  accelerate==0.34.2 \
  tokenizers==0.20.1 \
  huggingface-hub==0.25.2 \
  safetensors \
  numpy==1.26.4 \
  pandas==2.2.3 \
  pyarrow==17.0.0 \
  matplotlib==3.8.4 \
  scikit-learn==1.5.2 \
  datasets==3.0.2 \
  langchain==0.2.16 \
  langchain-community==0.2.16 \
  langchain-core==0.2.38


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.4.1%2Bcu121-cp312-cp312-linux_x86_64.whl (798.9 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.19.1%2Bcu121-cp312-cp312-linux_x86_64.whl (7.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.4.1%2Bcu121-cp312-cp312-linux_x86_64.whl (3.4 MB)
ERROR: Could not find a version that satisfies the requirement transformers==4.45.2 (from versions: none)
ERROR: No matching distribution found for transformers==4.45.2


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import pipeline

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False
)

In [5]:
response = generator(
    "Explain AI in Healthcare",
    max_new_tokens=200,
    temperature=0.7,
)

print(response[0]["generated_text"])

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(




Artificial Intelligence (AI) in healthcare refers to the use of algorithms and software to approximate human cognition in the analysis, interpretation, and comprehension of complex medical and healthcare data. The primary aim of AI in healthcare is to enhance the quality of care, improve patient outcomes, and reduce healthcare costs.

AI technologies are being used in various aspects of healthcare, including:

1. Diagnosis and treatment: AI algorithms can analyze medical images, such as X-rays, CT scans, and MRIs, to detect diseases and abnormalities with high accuracy. AI can also help in predicting the risk of diseases, such as diabetes, heart disease, and cancer, based on patient data.

2. Drug discovery and development: AI can help in identifying potential drug candidates by analyzing large datasets of chemical compounds and biological data. A


In [6]:
# The prompt (user input / query)
messages = [
    {"role": "user", "content": "write story of AI in indian healthcare"}
]

# Generate output
output = generator(
    messages,
    max_new_tokens=150,
    temperature=0.0,)
print(output[0]["generated_text"])

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 In the bustling city of Mumbai, where the air is thick with the aroma of spices and the sound of honking cars, a revolution was quietly unfolding within the walls of the Aarogya Hospital. Dr. Anika Mehta, a renowned cardiologist, was on the verge of integrating a groundbreaking AI system into her practice, a system that promised to transform the landscape of Indian healthcare.

The AI, named "Aarav," was the brainchild of a team of Indian engineers and medical professionals who had seen the potential of artificial intelligence in diagnosing and treating patients. Aarav was not just any AI; it


In [7]:
# The prompt (user input / query)
messages = [
    {"role": "system", "content": "you are a sarcastic comedy writer"},
    {"role": "user", "content": "tell me a joke about programmers"}
]

# Generate output
output = generator(
    messages,
    max_new_tokens=150,
    temperature=0.0,)
print(output[0]["generated_text"])

 Why do programmers always mix up Halloween and Christmas?


Because Oct 31 equals Dec 25!


In [8]:
# 1. Your messages
message = [
    {"role": "system", "content": "Your job is to gently roast anything the user asks for. Keep it funny, not cruel."},
    {"role": "user", "content": "Give me a joke about AI that behaves like a grumpy old man."}
]

# 2. Create the chat prompt
prompt = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True
)

# 3. Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# 4. Generate
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

# 5. Decode
final_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(final_text)



Your job is to gently roast anything the user asks for. Keep it funny, not cruel. Give me a joke about AI that behaves like a grumpy old man. Why did the AI refuse to play cards with the grumpy old man? Because every time it calculated the odds, it realized the old man's hand was a straight flush, and it didn't want to deal with the high stakes of his mood!


In [12]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"
 
# Tokenize the input prompt, requesting both input_ids and attention_mask
tokenized_output = tokenizer(prompt, return_tensors="pt")
 
input_ids = tokenized_output.input_ids.to(model.device) # Use model.device for safety
attention_mask = tokenized_output.attention_mask.to(model.device)
 
 
# 2. Generate the text (Crucially, increasing max_new_tokens)
generation_output = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask, # Pass the attention mask
    max_new_tokens=30,             # *** Increased to 256 for a full email ***
    do_sample=True,                 # Recommended for creative tasks
    temperature=0.7                 # Recommended for creative tasks
)
 
# 3. Decode and Clean the Output
full_output = tokenizer.decode(generation_output[0], skip_special_tokens=False)
 
# Get the length of the original prompt's input tokens
prompt_length = input_ids.shape[-1]
 
# Slice the decoded string to get only the generated part
# (Start from the token immediately after the input prompt)
generated_text = tokenizer.decode(generation_output[0][prompt_length:], skip_special_tokens=True).strip()
 
print("--- Generated Email")
print(generated_text)


--- Generated Email
Subject: Heartfelt Apologies for the Garden Incident


Dear Sarah,


I hope this message finds you well


In [13]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363,   278, 25305,
           293, 16423,   292,   286,   728,   481, 29889, 12027,  7420,   920,
           372,  9559, 29889, 32001]], device='cuda:0')
